# The Midwest underwater

A look at 2019 floods in South Dakota, USA

Elsa Culler  
Nate Quarderer  
2025-05-19

# Access streamflow data

One way to express how big a flood is by estimating how often larger
floods occur. For example, you might have heard news media talking about
a “100-year flood”.

In this notebook, you will write Python code to download and work with a
**time series** of streamflow data during the flooding on the Cheyenne
River.

> **Tip**
>
> A **time series** of data is taken at the same location but collected
> regularly or semi-regularly over time.

You will then use the data to assess when the flooding was at it’s
worst.

As an **extra challenge** you could consider how the values compared to
other years by computing the flood’s **return period**.

> **Tip**
>
> A **return period** is an estimate of how often you might expect to
> see a flood of at least a particular size. This does *NOT* mean an
> extreme flood “has” to occur within the return period, or that it
> couldn’t occur more than once. However, it does allow us to assess the
> probability that a sequence of floods would happen and evaluate
> whether or not we need to change forecasting tools or engineering
> standards to meet a new reality. For example, it would be really
> unusual to get three 100-year floods in a ten year period without some
> kind of underlying change in the climate.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Here are some resources from your text book you can review to learn
more:</p>
<ul>
<li><a
href="https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/">Introduction
to time-series data</a></li>
<li><a
href="https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/floods-return-period-and-probability/">Flood
return period and probability</a></li>
</ul></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Explain what data you will need to complete this analysis,
including:</p>
<ol type="1">
<li>What type or types of data do you need?</li>
<li>How many years of data do you think you need to compute the return
period of an extreme event like the <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="1"></span> <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span> floods?</li>
</ol></div></div>

YOUR ANSWER HERE

## STEP 0: Get set up to use Python

Use the cell below to add necessary **package imports** to this
notebook. It’s best to import everything in your very first code cell
because it helps folks who are reading your code to figure out where
everything comes from (mostly right now this is **you** in the future).
It’s *very* frustrating to try to figure out what packages need to be
installed to get some code to run.

> **Note**
>
> Our friend [the PEP-8 style guide has some things to say about
> imports](https://peps.python.org/pep-0008/#imports). In particular,
> your imports should be in alphabetical order.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>In the sample code below, we’ve imported a library needed for working
with <strong>tabular</strong>, or spreadsheet, data, as well as our own
library for common Environmental Data Analytics tasks (in this case,
managing files on your computer). You will also need to:</p>
<ol type="1">
<li>Add the <strong>library for working with vector data in
Python</strong> and a <strong>library for creating interactive plots of
vector and time-series data</strong> to the imports.</li>
<li>Check that your imports follow the PEP-8 guidelines – they should be
in alphabetical order.</li>
<li>Run your import cell to make sure everything will work</li>
</ol></div></div>

In [1]:
# Import libraries
import earthpy
import pandas as pd

In [2]:
# Import libraries
import earthpy
import geopandas as gpd # Vector data
import hvplot.pandas # Interactive plots
import pandas as pd

We like to keep important values up at the top of the notebook – it
makes them easy to modify. You can use the following cell to change
parameters about your workflow if you like:

In [3]:
id = 'stars'
site_name = 'Cheyenne River near Wasta'
year = 2019
project_title = 'Cheyenne River Flood Frequency'
project_dirname = 'flood-cheyenne'

Finally, we have arranged some sample data for you, which you can
download using the `earthpy` library. Later on, you’ll learn how to
download data from the NWIS using the `dataretrieval` library. For now,
you can use the sample data downloaded with the `earthpy` library.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>The following code will download the sample data based on the value
of “title”, and store it in the data directory on your computer. It will
also save the path to the downloaded data. You can use the
<code>project</code> later on to do things like locate data files on the
computer or image you’re using to code. You should practice writing
descriptive code by:</p>
<ol type="1">
<li>Change <code>'project-folder-name'</code> to a descriptive directory
name where you want to store your data.</li>
<li>Change <code>data_path</code> to a descriptive variable name</li>
<li>Run the data download cell to make sure everything will work</li>
</ol></div></div>

In [4]:
# Create project directory
project = earthpy.Project(title=project_title, dirname='project-folder-name')
# Download data
data_path = project.get_data()
# Display the project data directory location
project.project_dir

In [5]:
# Create project directory
project = earthpy.Project(title=project_title, dirname=project_dirname)
# Download data
streamflow_path = project.get_data()
# Display the project data directory location
project.project_dir

You can use an open science tool called `bash` or the `shell` to work
with files and get information about your file system. For example, this
code will **list** (ls) the contents of the project directory

In [6]:
!ls "$project.project_dir"

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Go check to see if you can find the files using some other
method!</p></div></div>

> **Warning**
>
> Are you working in the cloud, such as on GitHub Codespaces? Be aware
> that any files you download to a cloud computer **will not be saved**
> on the physical computer you are using! They will remain in the cloud.
> So, you will not be able to see any downloaded files using the File
> Explorer or Finder on your computer because they aren’t there.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Make sure to store the new version of your <code>DataFrame</code> for
other notebooks!</p>
<div id="383a0550" class="cell" data-execution_count="7">
<div class="sourceCode" id="cb1"><pre
class="sourceCode python cell-code"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="op">%</span>store project streamflow_path</span></code></pre></div>
</div></div></div>

## Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! This code will store all your variables. You might want to
specify specific variables, especially if you have large objects in
memory that you won’t need in the future.

In [8]:
%store

Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!